In [ ]:
import requests
import pandas as pd
import time

In [ ]:

# eBay API credentials
API_KEY = "Your OAuth TOKEN"
BASE_URL = "https://api.ebay.com/buy/browse/v1/item_summary/search"

# Headers with authentication
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "X-EBAY-C-MARKETPLACE-ID": "EBAY_US",
    "Content-Type": "application/json"
}

# Parameters for active gift card auctions
params = {
    "q": "Gift Card",
    "limit": "50",  
    "filter": "buyingOptions:{AUCTION},availability:{AVAILABLE},categoryIds:{172008}",
    "sort": "price DESC",
    "offset": 0
}

# Constants

# List to store all results
all_results = []
page = 1

print("🔍 Starting eBay auction scraping...")

# Pagination loop
while True:
    print(f"📄 Scraping page {page}...") #Usefull to ensure data scrapping entegrity

    response = requests.get(BASE_URL, headers=headers, params=params)
    print(response.status_code)

    if response.status_code == 200:
        data = response.json()
        items = data.get('itemSummaries', [])
        #print(items) to see the dict of categories and values included

        if not items:
            print("✅ No more items. Scraping completed.")
            break

        # Append data from the current page
        for item in items:
            all_results.append({
                "Seller_name": item.get('seller', {}).get('username'),
                "Feedback": item.get('seller', {}).get('feedbackScore'),
                "ID": item.get("itemId"),
                "Title": item.get("title"),
                "Value": item.get("price", {}).get("value"),
                "Shipping_cost": item.get('shippingOptions', [{}])[0].get('ShippingCost', {}).get('value'),
                "Currency": item.get("price", {}).get("currency"),
                "Сurrent Bid": item.get('currentBidPrice').get('value'),
                "Bid Count": item.get('bidCount'), 
                "End Time": item.get("itemEndDate"),
                "Condition": item.get("condition"),
                "URL": item.get("itemWebUrl")
            })

        # Move to the next page
        params["offset"] += len(items)
        page += 1
        
        # Avoid hitting rate limits
        time.sleep(1)  # 1-second delay between requests

    else:
        print(f"❌ Failed on page {page}: {response.status_code}, {response.text}")
        break

# Convert results to DataFrame and save to CSV
df = pd.DataFrame(all_results)
df['Amount'] = df['Title'].str.extract(r'\$(\d+)').astype(float) #Find value of each gift card
output_file = "ebay_gift_card_auctions.csv"
df.to_csv(output_file, index=False)

print(f"✅ Data saved to {output_file}")
